In [1]:
import os

In [2]:
%pwd

'd:\\TextSummarizer\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\TextSummarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# return type
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directory

In [18]:
class ConfigurationManager:
    def __init__(self,
                 Config_filePath = CONFIG_FILE_PATH,
                 Params_filePath = PARAMS_FILE_PATH,):
        self.config = read_yaml(Config_filePath)
        self.params = read_yaml(Params_filePath)
        create_directory([self.config.artifacts_root])
        
    def get_dataIngestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directory([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
             unzip_dir= config.unzip_dir
        )
        return data_ingestion_config

In [19]:
from urllib import request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_fileSize

In [20]:

class DataIngestion:
    def __init__(self, config = DataIngestionConfig):
        self.config = config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_fileSize(str(self.config.local_data_file))}") 
    def extract_zip_file(self):
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [24]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_dataIngestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[Time: 2023-06-25 05:21:19,677]
[logging level: INFO]
[module: common]
[message: yaml file: config\config.yaml loaded successfully!]
-----------------------------------
[Time: 2023-06-25 05:21:19,679]
[logging level: INFO]
[module: common]
[message: yaml file: params.yaml loaded successfully!]
-----------------------------------
[Time: 2023-06-25 05:21:19,681]
[logging level: INFO]
[module: common]
[message: created directory at: artifacts]
-----------------------------------
[Time: 2023-06-25 05:21:19,682]
[logging level: INFO]
[module: common]
[message: created directory at: artifacts/data_ingestion]
-----------------------------------
[Time: 2023-06-25 05:21:30,624]
[logging level: INFO]
[module: 3950748496]
[message: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070